In [2]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever

# Import the FlashRank-specific compressor
from langchain_community.document_compressors import FlashrankRerank

# Load environment variables
load_dotenv()
MINIML_MODEL_PATH = os.getenv("MINIML_MODEL_PATH")

# Sample text
sample_text = '''Artificial Intelligence refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions.
        AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer service (chatbots).
        Key types include Narrow AI (focused on specific tasks like voice assistants) and General AI (hypothetical AI capable of performing any intellectual task a human can do).
        '''



In [3]:
# 1. Create Document objects
doc_object = Document(page_content=sample_text, metadata={"source": "manual_input"})

# 2. Chunk the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
chunks = text_splitter.split_documents([doc_object])

print(f"Created {len(chunks)} chunks")

# 3. Initialize Embeddings
embeddings = HuggingFaceEmbeddings(model_name=MINIML_MODEL_PATH)

Created 3 chunks


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

In [1]:
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.documents import Document
from flashrank import Ranker, RerankRequest

# Load environment variables
load_dotenv()
MINIML_MODEL_PATH = os.getenv("MINIML_MODEL_PATH")

# Sample text
sample_text = '''Artificial Intelligence refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions.
        AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer service (chatbots).
        Key types include Narrow AI (focused on specific tasks like voice assistants) and General AI (hypothetical AI capable of performing any intellectual task a human can do).
        '''

# 1. Create Document objects
doc_object = Document(page_content=sample_text, metadata={"source": "manual_input"})

# 2. Chunk the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
chunks = text_splitter.split_documents([doc_object])

print(f"Created {len(chunks)} chunks")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk.page_content[:100]}...")

# 3. Initialize Embeddings
embeddings = HuggingFaceEmbeddings(model_name=MINIML_MODEL_PATH)

# 4. Create persist directory
current_dir = os.getcwd()
persist_directory = os.path.join(current_dir, "resources", "databases", "db1", "chroma_db")
os.makedirs(persist_directory, exist_ok=True)

# 5. Create Chroma Vector DB
print("\nCreating Chroma vector database...")
chroma_db = Chroma.from_documents(
    documents=chunks, 
    embedding=embeddings, 
    persist_directory=persist_directory
)
print("Chroma vector database created and stored locally.")



Created 3 chunks
Chunk 1: Artificial Intelligence refers to the simulation of human intelligence in machines that are programm...
Chunk 2: AI is used in fields such as healthcare (diagnostic tools), finance (fraud detection), and customer ...
Chunk 3: Key types include Narrow AI (focused on specific tasks like voice assistants) and General AI (hypoth...


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]


Creating Chroma vector database...
Chroma vector database created and stored locally.


In [9]:
# First, import the Ranker from flashrank
from flashrank import Ranker, RerankRequest

# Initialize the Ranker
ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")

# Now create the FlashrankRerank compressor
compressor = FlashrankRerank(
    ranker=ranker,  # Pass the initialized ranker
    top_n=3,
    score=True
)

# Alternative: Let FlashrankRerank initialize the ranker for you
# (This should also work after model_rebuild)
try:
    # Try the simple initialization first
    compressor = FlashrankRerank(
        model="ms-marco-MiniLM-L-12-v2",
        top_n=3,
        score=True
    )
except Exception as e:
    print(f"Simple initialization failed: {e}")
    print("Using explicit ranker initialization...")
    
    # If that fails, initialize with explicit ranker
    from flashrank import Ranker
    ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")
    compressor = FlashrankRerank(
        ranker=ranker,
        top_n=3,
        score=True
    )

# Create base retriever (retrieve more documents for reranking)
base_retriever = chroma_db.as_retriever(search_kwargs={"k": 10})

# Create compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# Test the reranker
print(f"Query: {query}")
reranked_docs = compression_retriever.invoke(query)  # Use invoke() instead of get_relevant_documents()

print(f"\nAfter FlashrankRerank (top 3 results):")
for i, doc in enumerate(reranked_docs):
    print(f"Result {i+1}: {doc.page_content[:150]}...")
    if hasattr(doc, 'metadata'):
        print(f"  Metadata: {doc.metadata}\n")

# 8. Alternative: Direct FlashRank implementation (simpler, no Pydantic issues)
print(f"\n--- Alternative: Direct FlashRank Implementation ---")

ValidationError: 2 validation errors for FlashrankRerank
ranker
  Extra inputs are not permitted [type=extra_forbidden, input_value=<flashrank.Ranker.Ranker object at 0x7acb7ff51270>, input_type=Ranker]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden
score
  Extra inputs are not permitted [type=extra_forbidden, input_value=True, input_type=bool]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden

In [ ]:
from flashrank import Ranker, RerankRequest

# Initialize FlashRank directly
direct_ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")

def rerank_with_flashrank(query, documents, top_k=3):
    """Direct reranking without LangChain wrapper"""
    
    # Prepare passages for FlashRank
    passages = []
    for i, doc in enumerate(documents):
        passages.append({
            "id": i,
            "text": doc.page_content,
            "metadata": doc.metadata
        })
    
    # Create rerank request
    rerank_request = RerankRequest(query=query, passages=passages)
    
    # Perform reranking
    results = direct_ranker.rerank(rerank_request)
    
    # Get top_k results
    top_results = results[:top_k]
    
    # Return reranked documents with scores
    reranked_docs = []
    for result in top_results:
        original_idx = result["id"]
        doc = documents[original_idx]
        # Add score to metadata
        doc.metadata["relevance_score"] = result["score"]
        doc.metadata["rank"] = result.get("index", 0)
        reranked_docs.append(doc)
    
    return reranked_docs

# Get initial documents
initial_docs = chroma_db.similarity_search(query, k=10)
print(f"Retrieved {len(initial_docs)} initial documents")

# Apply direct reranking
direct_results = rerank_with_flashrank(query, initial_docs, top_k=3)

print(f"\nDirect FlashRank results:")
for i, doc in enumerate(direct_results):
    score = doc.metadata.get('relevance_score', 'N/A')
    print(f"Result {i+1} (Score: {score:.4f}): {doc.page_content[:150]}...")

# 9. Comparison function
print(f"\n--- Comparison: Without vs With Reranking ---")

def compare_methods(query, k=3):
    print(f"Query: '{query}'")
    
    # Method 1: Without reranking
    without = chroma_db.similarity_search(query, k=k)
    print(f"\n1. WITHOUT RERANKING (Similarity Search):")
    for i, doc in enumerate(without):
        print(f"   {i+1}. {doc.page_content[:100]}...")
    
    # Method 2: With direct FlashRank reranking
    initial = chroma_db.similarity_search(query, k=10)
    with_rerank = rerank_with_flashrank(query, initial, top_k=k)
    print(f"\n2. WITH FLASHRANK RERANKING:")
    for i, doc in enumerate(with_rerank):
        score = doc.metadata.get('relevance_score', 'N/A')
        print(f"   {i+1}. Score: {score:.4f} - {doc.page_content[:100]}...")

# Run comparison
compare_methods(query)

# 10. Test with multiple queries
print(f"\n--- Testing Multiple Queries ---")

test_queries = [
    "What are the applications of AI in healthcare?",
    "Explain Narrow AI and General AI",
    "How do machines mimic human intelligence?",
]

for test_query in test_queries:
    print(f"\n{'-'*50}")
    print(f"Query: '{test_query}'")
    
    # Get initial results
    initial = chroma_db.similarity_search(test_query, k=8)
    
    # Rerank
    reranked = rerank_with_flashrank(test_query, initial, top_k=2)
    
    print("Top 2 reranked results:")
    for i, doc in enumerate(reranked):
        score = doc.metadata.get('relevance_score', 'N/A')
        print(f"{i+1}. (Score: {score:.4f}) {doc.page_content[:120]}...")

# 11. Custom retriever class (using direct FlashRank)
print(f"\n--- Custom Retriever with FlashRank ---")

from langchain_core.retrievers import BaseRetriever
from typing import List

class FlashRankRetriever(BaseRetriever):
    """Custom retriever with built-in FlashRank reranking"""
    
    def __init__(self, vectorstore, k_final=3, k_initial=10, model="ms-marco-MiniLM-L-12-v2"):
        super().__init__()
        self.vectorstore = vectorstore
        self.k_final = k_final
        self.k_initial = k_initial
        self.ranker = Ranker(model_name=model)
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        # Get initial documents
        initial_docs = self.vectorstore.similarity_search(query, k=self.k_initial)
        
        # Prepare for reranking
        passages = [{"text": doc.page_content, "metadata": doc.metadata} 
                   for doc in initial_docs]
        
        # Rerank
        rerank_request = RerankRequest(query=query, passages=passages)
        results = self.ranker.rerank(rerank_request)
        
        # Return top k_final documents
        reranked_docs = []
        for result in results[:self.k_final]:
            doc = initial_docs[result["id"]]
            doc.metadata["relevance_score"] = result["score"]
            reranked_docs.append(doc)
        
        return reranked_docs

# Test custom retriever
custom_retriever = FlashRankRetriever(
    vectorstore=chroma_db,
    k_final=2,
    k_initial=8
)

print(f"Testing custom retriever with query: '{query}'")
custom_results = custom_retriever._get_relevant_documents(query)
for i, doc in enumerate(custom_results):
    score = doc.metadata.get('relevance_score', 'N/A')
    print(f"Custom result {i+1} (Score: {score:.4f}): {doc.page_content[:100]}...")

print("\n✅ Implementation complete!")